In [1]:
import jsonlines
import torchaudio
from pathlib import Path
import torch, random
import librosa, os
import IPython.display as ipd
from dotenv import load_dotenv
from spellchecker import SpellChecker
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union
import pandas as pd
from sklearn.model_selection import train_test_split
import cv2
from collections import defaultdict
from torch.utils.data import Dataset
from PIL import Image
import torchvision.transforms as transforms
from torch.utils.data import Dataset
from PIL import Image
import torchvision.transforms as transforms
import numpy as np
from skimage import io
from skimage.transform import resize
import matplotlib.pyplot as plt
import random
import matplotlib.patches as patches
import os

import torch
import torchvision
from torchvision import ops
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils.rnn import pad_sequence

In [2]:
import jsonlines
import pandas as pd

data = {'image': [], 'caption': [], 'bbox': []}
load_dotenv()

TEAM_NAME = os.getenv("TEAM_NAME", "7up")
TEAM_TRACK = os.getenv("TEAM_TRACK", "advanced")


input_dir = Path(f"/home/jupyter/{TEAM_TRACK}")
with jsonlines.open(input_dir / "vlm.jsonl") as reader:
    for obj in reader:
        image = obj['image']
        annotations = obj['annotations']
        for annotation in annotations:
            caption = annotation['caption']
            bbox = annotation['bbox']
            data['image'].append(image)
            data['caption'].append(caption)
            data['bbox'].append(bbox)

df = pd.DataFrame(data)
grouped_df = df.groupby('image').agg(list).reset_index()
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)
grouped_df.head()

,image,caption,bbox
0,image_0.jpg,"[grey missile, red, white, and blue light airc...","[[912, 164, 48, 152], [1032, 80, 24, 28], [704..."
1,image_1.jpg,"[grey camouflage fighter jet, grey and white f...","[[1112, 172, 64, 36], [1108, 512, 144, 48], [3..."
2,image_10.jpg,"[blue and grey fighter jet, blue helicopter, b...","[[792, 208, 52, 40], [964, 392, 52, 52], [408,..."
3,image_100.jpg,"[grey camouflage fighter jet, white, black, an...","[[1108, 224, 56, 44], [788, 148, 92, 32], [516..."
4,image_1000.jpg,"[red and white fighter plane, yellow commercia...","[[412, 252, 68, 36], [544, 276, 44, 36], [1092..."


In [3]:
unique_values = df['caption'].unique()
id_list = [i for i in unique_values]
name2idx = {value: idx for idx, value in enumerate(unique_values)}
idx2name = {v:k for k, v in name2idx.items()}

In [4]:
id_list

['grey missile',
 'red, white, and blue light aircraft',
 'green and black missile',
 'white and red helicopter',
 'grey camouflage fighter jet',
 'grey and white fighter plane',
 'white and black drone',
 'white and black fighter jet',
 'white missile',
 'black and white commercial aircraft',
 'grey drone',
 'yellow, red, and grey helicopter',
 'yellow commercial aircraft',
 'black cargo aircraft',
 'yellow helicopter',
 'white and black light aircraft',
 'grey and black fighter plane',
 'red fighter plane',
 'blue helicopter',
 'white, red, and green fighter plane',
 'black camouflage fighter jet',
 'green light aircraft',
 'blue and yellow fighter jet',
 'white fighter jet',
 'red fighter jet',
 'blue and white light aircraft',
 'white and black helicopter',
 'white and blue fighter plane',
 'grey commercial aircraft',
 'blue and grey fighter jet',
 'green and brown camouflage fighter jet',
 'red and grey missile',
 'red and white fighter jet',
 'orange light aircraft',
 'yellow lig

In [5]:
count = 0
for _, row in grouped_df.iterrows():
    if count == 5:
        break
    x, y, z = row
    print(row)
    count += 1
    

image                                            image_0.jpg
caption    [grey missile, red, white, and blue light airc...
bbox       [[912, 164, 48, 152], [1032, 80, 24, 28], [704...
Name: 0, dtype: object
image                                            image_1.jpg
caption    [grey camouflage fighter jet, grey and white f...
bbox       [[1112, 172, 64, 36], [1108, 512, 144, 48], [3...
Name: 1, dtype: object
image                                           image_10.jpg
caption    [blue and grey fighter jet, blue helicopter, b...
bbox       [[792, 208, 52, 40], [964, 392, 52, 52], [408,...
Name: 2, dtype: object
image                                          image_100.jpg
caption    [grey camouflage fighter jet, white, black, an...
bbox       [[1108, 224, 56, 44], [788, 148, 92, 32], [516...
Name: 3, dtype: object
image                                         image_1000.jpg
caption    [red and white fighter plane, yellow commercia...
bbox       [[412, 252, 68, 36], [544, 276, 44, 36], [1

In [6]:
import os

def convert_bbox_to_yolo(img_width, img_height, tlwh_bbox):
    x_tl, y_tl, w, h = tlwh_bbox
    cx = x_tl + w / 2.0
    cy = y_tl + h / 2.0
    return [cx / img_width, cy / img_height, w / img_width, h / img_height]

base_path = "dataset_vlm"

label_train_path = os.path.join(base_path, "labels")
os.makedirs(label_train_path, exist_ok=True)
img_width = 1520
img_height = 870

for _, row in grouped_df.iterrows():
    filename = row['image']
    bboxes = row['bbox']
    captions = row['caption']

    label_path = os.path.join(label_train_path, filename.replace(".jpg", ".txt"))

    with open(label_path, 'w') as f:
        for bbox, caption in zip(bboxes, captions):
            yolo_bbox = convert_bbox_to_yolo(img_width, img_height, bbox)
            class_id = name2idx[caption]
            # Write bbox to label file
            f.write(f"{class_id} {yolo_bbox[0]:.6f} {yolo_bbox[1]:.6f} {yolo_bbox[2]:.6f} {yolo_bbox[3]:.6f}\n")

print("Dataset preparation complete.")

Dataset preparation complete.
